In [2]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Quadro RTX 6000')

In [3]:
import os
os.chdir("/home/TFM_Laura_Monne")
os.getcwd()


'/home/TFM_Laura_Monne'

In [ ]:
!python quantus_evaluation.py \
    --model_path results/best_model.pth \
    --data_dir ./data \
    --num_samples 5 \
    --methods gradcam gradcampp integrated_gradients saliency \
    --output outputs/quantus_metrics.json

  EVALUACIÓN QUANTUS - RESNET18 XAI
Dispositivo: cuda
Métodos: ['gradcam', 'gradcampp', 'integrated_gradients', 'saliency']
Muestras a evaluar: 5
Cargando modelo desde results/best_model.pth...
Modelo ResNet-18 creado:
  - Parámetros totales: 22,362,142
  - Parámetros entrenables: 22,362,142
  - Número de clases: 15
✅ Modelo cargado correctamente.
Cargando datasets de MedMNIST...

Cargando BLOODMNIST...
  - Entrenamiento: 11959
  - Validación:    1712
  - Test:          3421
  - Clases:        8
  - Canales (orig):3 -> input: 3

Cargando RETINAMNIST...
  - Entrenamiento: 1080
  - Validación:    120
  - Test:          400
  - Clases:        5
  - Canales (orig):3 -> input: 3

Cargando BREASTMNIST...
  - Entrenamiento: 546
  - Validación:    78
  - Test:          156
  - Clases:        2
  - Canales (orig):1 -> input: 3
DataLoaders creados:
  - Train: 13585 muestras, 13585 batches
  - Val:   1910 muestras, 1910 batches
  - Test:  3977 muestras, 3977 batches
Recolectando muestras:   0%|  